In [5]:
import matplotlib.pyplot as plt
import flightTools as ft
import numpy as np
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense

In [6]:
class FlightController:

    # This class contains the AI that the rocket will use to
    # control the throttle

    def __init__(self, weights=None):

        # Construct the model below using
        # model.add(...)
        self.model = Sequential()
        self.model.add(Dense(3, input_shape=(3,), activation='sigmoid'))
        self.model.add(Dense(1, activation='sigmoid'))

    def get_weights(self):
        return self.model.get_weights()

    def reset_weights(self):
        shapes = [w.shape for w in self.model.get_weights()]
        new_weights = []
        for s in shapes:
            new_weights.append(np.random.uniform(-1.0, 1.0, size=s))
        self.model.set_weights(new_weights)
        
    def __call__(self, flight):
        
        x = flight.position[-1][1]
        v = flight.velocity[-1][1]
        
        data = np.array([[x, v]])
        
        return self.model.predict(data)[0][0]

In [9]:
def perform_run():
    
    fc = FlightController()
    fc.reset_weights()
    flight = ft.Flight(fc, verbose=False)
    flight.run()

    return fc.get_weights(), flight

In [11]:
weights = []
flights = []

for i in tqdm(range(1000)):
    w, f = perform_run()
    weights.append(w), flights.append(f)

 35%|███▌      | 353/1000 [09:07<16:42,  1.55s/it]

KeyboardInterrupt: 